<a href="https://colab.research.google.com/github/aneeq-shaffy/DL-labsheets/blob/main/Pest_solution_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import EfficientNetB0
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import train_test_split



In [3]:
import kagglehub

path = kagglehub.dataset_download("imbikramsaha/paddy-doctor")
print("Dataset path:", path)


Using Colab cache for faster access to the 'paddy-doctor' dataset.
Dataset path: /kaggle/input/paddy-doctor


In [4]:
dataset_root_path = os.path.join(path, 'paddy-disease-classification')
train_csv_path = os.path.join(dataset_root_path, 'train.csv')
train_images_path = os.path.join(dataset_root_path, 'train_images')


In [5]:
full_df = pd.read_csv(train_csv_path)


In [6]:
print(full_df.columns)


Index(['image_id', 'label', 'variety', 'age'], dtype='object')


In [7]:
labels = sorted(full_df["label"].unique())
print(labels)


['bacterial_leaf_blight', 'bacterial_leaf_streak', 'bacterial_panicle_blight', 'blast', 'brown_spot', 'dead_heart', 'downy_mildew', 'hispa', 'normal', 'tungro']


In [8]:
import os

dataset_path = "/kaggle/input/paddy-doctor"

for root, dirs, files in os.walk(dataset_path):
    level = root.replace(dataset_path, '').count(os.sep)
    indent = ' ' * 4 * level
    print(f"{indent}{os.path.basename(root)}/")
    subindent = ' ' * 4 * (level + 1)
    for f in files[:5]:  # only first 5 files
        print(f"{subindent}{f}")


paddy-doctor/
    paddy-disease-classification/
        sample_submission.csv
        .jovianrc
        train.csv
        train_images/
            tungro/
                109629.jpg
                104765.jpg
                109706.jpg
                100098.jpg
                102734.jpg
            hispa/
                100502.jpg
                107167.jpg
                106262.jpg
                102866.jpg
                104880.jpg
            downy_mildew/
                101119.jpg
                105381.jpg
                110270.jpg
                110143.jpg
                105940.jpg
            bacterial_leaf_streak/
                103494.jpg
                106019.jpg
                103874.jpg
                103325.jpg
                109858.jpg
            bacterial_leaf_blight/
                109940.jpg
                105979.jpg
                104324.jpg
                109428.jpg
                106615.jpg
            brown_spot/
                104821.jpg
   

In [9]:
train_df, val_df = train_test_split(
    full_df,
    test_size=0.1,
    stratify=full_df['label'],
    random_state=42
)

print(len(train_df), len(val_df))


9366 1041


In [10]:
import numpy as np

label_counts = train_df['label'].value_counts()
label_order = sorted(train_df['label'].unique())

counts = np.array([label_counts[label] for label in label_order], dtype=float)
class_weights = 1.0 / counts
class_weights = class_weights / class_weights.sum()

# Keras expects a dictionary: {class_index: weight}
class_index_mapping = {label: idx for idx, label in enumerate(label_order)}
class_weights_dict = {class_index_mapping[label]: weight for label, weight in zip(label_order, class_weights)}

print(class_weights_dict)


{0: np.float64(0.1521780416341574), 1: np.float64(0.19177992966175975), 2: np.float64(0.21646447506376845), 3: np.float64(0.04193653193370962), 4: np.float64(0.07556305984368876), 5: np.float64(0.05053061320826028), 6: np.float64(0.11754253753462696), 7: np.float64(0.045706436198133686), 8: np.float64(0.041302730443527606), 9: np.float64(0.06699564447836756)}


In [11]:
import tensorflow as tf
import numpy as np

IMG_SIZE = 224
BATCH_SIZE = 32
DATA_DIR = "/kaggle/input/paddy-doctor/paddy-disease-classification/train_images"
EPOCHS = 10


In [12]:
train_dataset = tf.keras.utils.image_dataset_from_directory(
    DATA_DIR,
    image_size=(224,224),
    batch_size=32,
    label_mode="int",          # 🔥 CHANGE THIS
    validation_split=0.1,
    subset="training",
    seed=42
)

val_dataset = tf.keras.utils.image_dataset_from_directory(
    DATA_DIR,
    image_size=(224,224),
    batch_size=32,
    label_mode="int",          # 🔥 CHANGE THIS
    validation_split=0.1,
    subset="validation",
    seed=42
)

class_names = train_dataset.class_names


Found 10407 files belonging to 10 classes.
Using 9367 files for training.
Found 10407 files belonging to 10 classes.
Using 1040 files for validation.


In [13]:
from tensorflow.keras.applications.efficientnet import preprocess_input

train_dataset = train_dataset.map(
    lambda x, y: (preprocess_input(x), y),
    num_parallel_calls=tf.data.AUTOTUNE
)

val_dataset = val_dataset.map(
    lambda x, y: (preprocess_input(x), y),
    num_parallel_calls=tf.data.AUTOTUNE
)


In [14]:
import numpy as np

label_counts = {i: 0 for i in range(len(class_names))}

for _, labels in train_dataset.unbatch():
    label_counts[int(labels.numpy())] += 1

class_weights_dict = {
    i: max(label_counts.values()) / count
    for i, count in label_counts.items()
}

print(class_weights_dict)


{0: 3.6492027334851938, 1: 4.711764705882353, 2: 5.322259136212624, 3: 1.0203821656050955, 4: 1.8287671232876712, 5: 1.237065637065637, 6: 2.907441016333938, 7: 1.132155477031802, 8: 1.0, 9: 1.638036809815951}


In [ ]:
from tensorflow.keras.applications import EfficientNetB0
from tensorflow.keras import layers, models

num_classes = len(class_names)

base_model = EfficientNetB0(
    include_top=False,
    input_shape=(IMG_SIZE, IMG_SIZE, 3),
    weights='imagenet'
)

# ----------------------------
# Step 1: Train head first
# ----------------------------
base_model.trainable = False  # freeze backbone

inputs = tf.keras.Input(shape=(IMG_SIZE, IMG_SIZE, 3))
x = base_model(inputs, training=False)
x = layers.GlobalAveragePooling2D()(x)
x = layers.Dropout(0.3)(x)
outputs = layers.Dense(num_classes, activation='softmax')(x)

model = models.Model(inputs, outputs)

# Compile for head training
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=1e-3),
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

# Train head first
history_head = model.fit(
    train_dataset,
    validation_data=val_dataset,
    epochs=5
)

# ----------------------------
# Step 2: Fine-tune top layers
# ----------------------------
base_model.trainable = True
for layer in base_model.layers[:-20]:
    layer.trainable = False

model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=1e-4),
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

history_fine = model.fit(
    train_dataset,
    validation_data=val_dataset,
    epochs=5
)


16705208/16705208 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
Epoch 1/5
261/293 ━━━━━━━━━━━━━━━━━━━━ 1:08 2s/step - accuracy: 0.3563 - loss: 1.8732

In [ ]:
history = model.fit(
    train_dataset,
    validation_data=val_dataset,
    epochs=5
)


In [ ]:
import tensorflow as tf

# 1️⃣ Save the trained Keras model (optional)
model.save("pest_disease_detection_model.h5")  # HDF5 backup

# 2️⃣ Convert to TensorFlow Lite
converter = tf.lite.TFLiteConverter.from_keras_model(model)

# Optional optimization
converter.optimizations = [tf.lite.Optimize.DEFAULT]

tflite_model = converter.convert()

# 3️⃣ Save as TFLite
with open("pest_disease_detection_model.tflite", "wb") as f:
    f.write(tflite_model)

print("TFLite model saved as pest_disease_detection_model.tflite")


In [ ]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("tntiphan/paddy-rice-disease-classification")

print("Path to dataset files:", path)

In [ ]:
import tensorflow as tf
import numpy as np

# 1️⃣ Load the TFLite model
tflite_model_path = "/content/pest_disease_detection_model.tflite"
interpreter = tf.lite.Interpreter(model_path=tflite_model_path)
interpreter.allocate_tensors()

# 2️⃣ Get input and output details
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

input_shape = input_details[0]['shape']
print(f"Model input shape: {input_shape}")

# 3️⃣ Function to predict a single image
def tflite_predict(image):
    # Resize and normalize image
    img = tf.image.resize(image, (input_shape[1], input_shape[2]))
    img = tf.expand_dims(img, axis=0)  # batch dimension
    img = tf.cast(img, tf.float32) / 255.0  # normalization

    interpreter.set_tensor(input_details[0]['index'], img.numpy())
    interpreter.invoke()
    output = interpreter.get_tensor(output_details[0]['index'])
    return np.argmax(output, axis=1)[0]

# Example usage:
# pred_class = tflite_predict(some_image_tensor)
# print("Predicted class index:", pred_class)


In [ ]:
import os

dataset_path = "/root/.cache/kagglehub/datasets/tntiphan/paddy-rice-disease-classification/versions/7"

for root, dirs, files in os.walk(dataset_path):
    level = root.replace(dataset_path, '').count(os.sep)
    indent = ' ' * 4 * level
    print(f"{indent}{os.path.basename(root)}/")

    subindent = ' ' * 4 * (level + 1)
    # Only show first 5 files
    for f in files[:5]:
        print(f"{subindent}{f}")
